# Regression

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [3]:
import statsmodels.api as sm
import joblib
import os

In [4]:
df = pd.read_csv('Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv')

In [5]:
df.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
0,10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0.0
1,6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0.0
2,7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0.0
3,9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0.0
4,2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0.0


In [6]:
male_df = df[~(df['segment'] == 'Womens E-Mail')]
male_df['treatment'] = male_df['segment'].apply(lambda x: 1 if x == 'Mens E-Mail' else 0)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [58]:
# treatment_df_tmp = male_df[male_df['treatment'] == 1]
# control_df_tmp = male_df[male_df['treatment'] == 0]

# treatment_df = treatment_df_tmp.drop(treatment_df_tmp[~(
#     (treatment_df_tmp['history'] > 300) |
#     (treatment_df_tmp['recency'] < 6) |
#     (treatment_df_tmp['channel'] == 'Multichannel')
# )].sample(frac=0.5, random_state=1).index)

# control_df = control_df_tmp.drop(control_df_tmp[
#     (control_df_tmp['history'] > 300) |
#     (control_df_tmp['recency'] < 6) |
#     (control_df_tmp['channel'] == 'Multichannel')
# ].sample(frac=0.5, random_state=1).index)
# biased_df = pd.concat([treatment_df, control_df])

sample_rules = (male_df.history > 300) | (male_df.recency < 6) | (male_df.channel=='Multichannel')
biased_df = pd.concat([
    male_df[(sample_rules) & (male_df.treatment == 0)].sample(frac=0.5, random_state=1),
    male_df[(sample_rules) & (male_df.treatment == 1)],
    male_df[(~sample_rules) & (male_df.treatment == 0)],
    male_df[(~sample_rules) & (male_df.treatment == 1)].sample(frac=0.5, random_state=1)
], axis=0, ignore_index=True)

biased_df.head()

,recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend,treatment
0,8,5) $500 - $750,572.65,1,0,Urban,1,Web,No E-Mail,0,0,0.0,0
1,5,1) $0 - $100,42.38,1,0,Urban,1,Phone,No E-Mail,1,0,0.0,0
2,1,"7) $1,000 +",3003.48,1,1,Urban,1,Phone,No E-Mail,0,0,0.0,0
3,1,5) $500 - $750,662.10,0,1,Urban,1,Web,No E-Mail,0,0,0.0,0
4,5,1) $0 - $100,44.37,0,1,Urban,0,Web,No E-Mail,0,0,0.0,0


In [59]:
# 回帰分析の実行
X = biased_df[['treatment', 'history']]
X = sm.add_constant(X) # statsmodelsではβ0を明示的に入れてあげる必要がある
y = biased_df['spend']
model = sm.OLS(y, X)
biased_reg = model.fit()

In [60]:
# 回帰分析結果のレポート
biased_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  spend   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     20.45
Date:                Sat, 21 Nov 2020   Prob (F-statistic):           1.32e-09
Time:                        05:59:22   Log-Likelihood:            -1.3312e+05
No. Observations:               31925   AIC:                         2.663e+05
Df Residuals:                   31922   BIC:                         2.663e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.3413      0.147      2.327      0.020       0.054       0.629
treatment      0.9088      0.177      5.122      0.000       0.561       1.257
history        0.0011      0.000      3.096      0.002       0.000       0.002
==============================================================================
Omnibus:                    70760.532   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        352134568.791
Skew:                          20.807   Prob(JB):                         0.00
Kurtosis:                     515.825   Cond. No.                         833.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## 2.2 回帰分析におけるバイアス
### 2.2.1 共変量追加における効果

In [61]:
# RCTデータでの単回帰分析の実行
X = male_df['treatment']
X = sm.add_constant(X) # statsmodelsではβ0を明示的に入れてあげる必要がある
y = male_df['spend']
model = sm.OLS(y, X)
male_reg = model.fit()

In [62]:
# 回帰分析結果のレポート
male_reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  spend   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     28.09
Date:                Sat, 21 Nov 2020   Prob (F-statistic):           1.16e-07
Time:                        05:59:43   Log-Likelihood:            -1.7584e+05
No. Observations:               42613   AIC:                         3.517e+05
Df Residuals:                   42611   BIC:                         3.517e+05
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6528      0.103      6.356      0.000       0.451       0.854
treatment      0.7698      0.145      5.300      0.000       0.485       1.055
==============================================================================
Omnibus:                    94877.860   Durbin-Watson:                   2.000
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        502270597.344
Skew:                          21.023   Prob(JB):                         0.00
Kurtosis:                     533.203   Cond. No.                         2.62
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [65]:
# バイアスのあるデータでの単回帰分析の実行
X = biased_df['treatment']
X = sm.add_constant(X) # statsmodelsではβ0を明示的に入れてあげる必要がある
y = biased_df['spend']
model = sm.OLS(y, X)
biased_reg = model.fit()

In [68]:
# 回帰分析結果のレポート
biased_reg_table = biased_reg.summary()
## 推定されたパラメーターの取り出し
biased_reg_table.tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.5580,0.129,4.328,0.000,0.305,0.811
treatment,0.9837,0.176,5.596,0.000,0.639,1.328


In [69]:
# バイアスのあるデータでの重回帰分析の実行
biased_df['channel'] = pd.get_dummies(biased_df['channel'])
X = biased_df[['treatment', 'recency', 'channel', 'history']]
X = sm.add_constant(X) # statsmodelsではβ0を明示的に入れてあげる必要がある
y = biased_df['spend']
model = sm.OLS(y, X)
biased_mreg = model.fit()

In [70]:
# 回帰分析結果のレポート
biased_mreg_table = biased_mreg.summary()
## 推定されたパラメーターの取り出し
biased_mreg_table.tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,0.6008,0.240,2.500,0.012,0.130,1.072
treatment,0.8614,0.181,4.748,0.000,0.506,1.217
recency,-0.0362,0.026,-1.376,0.169,-0.088,0.015
channel,-0.1240,0.295,-0.420,0.675,-0.703,0.455
history,0.0010,0.000,2.656,0.008,0.000,0.002


### 2.2.2 Omitted Variable Bias

In [76]:
# model A: historyなしのデータで重回帰分析
X = biased_df[['treatment', 'recency', 'channel']]
X = sm.add_constant(X) # statsmodelsではβ0を明示的に入れてあげる必要がある
y = biased_df['spend']
model_a = sm.OLS(y, X)
biased_mreg_a = model_a.fit()
short_coef = biased_mreg_a.summary().tables[1]
print(short_coef)
short_coef_df = pd.read_html(short_coef.as_html(), header=0, index_col=0)[0] #SimpleTableは扱いなれてないのでpandas DataFrameにする

## aの結果から介入効果に関するパラメーターのみを取り出す
alpha_1 = biased_mreg_a.params['treatment'] # summaryのデータは小数点が四捨五入されているため、正確な値をとってくる
print(alpha_1)

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.8774      0.217      4.052      0.000       0.453       1.302
treatment      0.8902      0.181      4.915      0.000       0.535       1.245
recency       -0.0507      0.026     -1.969      0.049      -0.101      -0.000
channel        0.1878      0.271      0.693      0.489      -0.344       0.719
0.89017989812997


In [77]:
# model B: historyありのデータで重回帰分析
X = biased_df[['treatment', 'recency', 'channel', 'history']]
X = sm.add_constant(X) # statsmodelsではβ0を明示的に入れてあげる必要がある
y = biased_df['spend']
model_b = sm.OLS(y, X)
biased_mreg_b = model_b.fit()
long_coef = biased_mreg_b.summary().tables[1]
print(long_coef)
long_coef_df = pd.read_html(long_coef.as_html(), header=0, index_col=0)[0] #SimpleTableは扱いなれてないのでpandas DataFrameにする

## bの結果から介入とhistoryに関するパラメーターを取り出す
beta_1 = biased_mreg_b.params['treatment']
beta_2 = biased_mreg_b.params['history']

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.6008      0.240      2.500      0.012       0.130       1.072
treatment      0.8614      0.181      4.748      0.000       0.506       1.217
recency       -0.0362      0.026     -1.376      0.169      -0.088       0.015
channel       -0.1240      0.295     -0.420      0.675      -0.703       0.455
history        0.0010      0.000      2.656      0.008       0.000       0.002


/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [78]:
# model C: historyに対してモデルAの変数を使って重回帰分析
X = biased_df[['treatment', 'recency', 'channel']]
X = sm.add_constant(X) # statsmodelsではβ0を明示的に入れてあげる必要がある
y = biased_df['history']
model_c = sm.OLS(y, X)
biased_mreg_c = model_c.fit()
omitted_coef = biased_mreg_c.summary().tables[1]
omitted_coef_df = pd.read_html(omitted_coef.as_html(), header=0, index_col=0)[0] #SimpleTableは扱いなれてないのでpandas DataFrameにする
gamma_1 = biased_mreg_c.params['treatment']
print(omitted_coef)

                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        272.7624      3.174     85.939      0.000     266.541     278.983
treatment     28.4220      2.655     10.707      0.000      23.219      33.625
recency      -14.2853      0.377    -37.861      0.000     -15.025     -13.546
channel      307.4372      3.973     77.376      0.000     299.649     315.225


/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:2495: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


In [75]:
## OVBの確認
print(beta_2 * gamma_1)
print(alpha_1 - beta_1)

0.02882221175440766
0.028822211754403693


### 2.2.7 Post treatment bias

In [79]:
# Post treatment biasを使って重回帰分析を使って検証
X = biased_df[['treatment', 'visit', 'recency', 'channel', 'history']]
X = sm.add_constant(X) # statsmodelsではβ0を明示的に入れてあげる必要がある
y = biased_df['spend']
model_c = sm.OLS(y, X)
biased_mreg_ptb = model_c.fit()
biased_mreg_ptb.summary().tables[1]

,coef,std err,t,P>|t|,[0.025,0.975]
const,-0.2988,0.239,-1.250,0.211,-0.767,0.170
treatment,0.2783,0.180,1.545,0.122,-0.075,0.631
visit,7.2379,0.246,29.403,0.000,6.755,7.720
recency,0.0090,0.026,0.346,0.729,-0.042,0.060
channel,-0.1070,0.291,-0.367,0.714,-0.678,0.464
history,0.0005,0.000,1.316,0.188,-0.000,0.001
